# 📱 Klasifikasi Kinerja Smartphone

Notebook ini membangun model klasifikasi untuk menentukan:
1. **Kelas Harga**: Entry Level, Mid Range, High End, Flagship
2. **Jenis Penggunaan**: Gaming, Daily Use, Photography, Business, All-Rounder

**Dataset**: smartphones.csv (981 data smartphone)

**Algoritma**: Random Forest dengan anti-overfitting

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✅ Libraries imported successfully!")

## 1. Load Dataset

In [ ]:
# Load dataset
df = pd.read_csv('../dataset/smartphones.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Dataset info
print("Dataset Info:")
print("=" * 50)
df.info()

In [ ]:
# Check missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
print("Missing Values:")
print(missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False))

In [ ]:
# Statistical summary
df.describe()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Price distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['price'], bins=50, color='steelblue', edgecolor='white')
axes[0].set_xlabel('Price (₹)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Smartphone Prices')

# Box plot
axes[1].boxplot(df['price'].dropna())
axes[1].set_ylabel('Price (₹)')
axes[1].set_title('Price Box Plot')

plt.tight_layout()
plt.show()

In [ ]:
# Brand distribution
brand_counts = df['brand_name'].value_counts().head(15)

plt.figure(figsize=(12, 6))
brand_counts.plot(kind='bar', color='steelblue', edgecolor='white')
plt.xlabel('Brand')
plt.ylabel('Count')
plt.title('Top 15 Smartphone Brands')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Processor brand distribution
processor_counts = df['processor_brand'].value_counts()

plt.figure(figsize=(10, 6))
processor_counts.plot(kind='pie', autopct='%1.1f%%', colors=plt.cm.Set3.colors)
plt.title('Processor Brand Distribution')
plt.ylabel('')
plt.tight_layout()
plt.show()

In [ ]:
# Key features scatter plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# RAM vs Price
axes[0, 0].scatter(df['ram_capacity'], df['price'], alpha=0.5, c='steelblue')
axes[0, 0].set_xlabel('RAM (GB)')
axes[0, 0].set_ylabel('Price (₹)')
axes[0, 0].set_title('RAM vs Price')

# Processor Speed vs Price
axes[0, 1].scatter(df['processor_speed'], df['price'], alpha=0.5, c='green')
axes[0, 1].set_xlabel('Processor Speed (GHz)')
axes[0, 1].set_ylabel('Price (₹)')
axes[0, 1].set_title('Processor Speed vs Price')

# Primary Camera vs Price
axes[1, 0].scatter(df['primary_camera_rear'], df['price'], alpha=0.5, c='orange')
axes[1, 0].set_xlabel('Primary Camera (MP)')
axes[1, 0].set_ylabel('Price (₹)')
axes[1, 0].set_title('Primary Camera vs Price')

# Refresh Rate vs Price
axes[1, 1].scatter(df['refresh_rate'], df['price'], alpha=0.5, c='purple')
axes[1, 1].set_xlabel('Refresh Rate (Hz)')
axes[1, 1].set_ylabel('Price (₹)')
axes[1, 1].set_title('Refresh Rate vs Price')

plt.tight_layout()
plt.show()

In [ ]:
# 5G distribution
has_5g_counts = df['has_5g'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart 5G
axes[0].pie(has_5g_counts, labels=['5G', 'Non-5G'], autopct='%1.1f%%', 
           colors=['#2ecc71', '#e74c3c'], startangle=90)
axes[0].set_title('5G Support Distribution')

# Price comparison 5G vs Non-5G
df.boxplot(column='price', by='has_5g', ax=axes[1])
axes[1].set_xlabel('Has 5G')
axes[1].set_ylabel('Price (₹)')
axes[1].set_title('Price Comparison: 5G vs Non-5G')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 3. Data Preprocessing

In [ ]:
# Create a copy for preprocessing
df_clean = df.copy()

# Drop rows with missing price (target variable)
df_clean = df_clean.dropna(subset=['price'])
print(f"After dropping missing prices: {len(df_clean)} rows")

# Fill missing values
# Numeric columns - fill with median
numeric_cols = ['rating', 'processor_speed', 'fast_charging', 'num_front_cameras']
for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Categorical columns - fill with mode
categorical_cols = ['processor_brand', 'os']
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

# Extended memory - fill with 0 if not available
df_clean['extended_upto'] = df_clean['extended_upto'].fillna(0)

print(f"\nMissing values after cleaning:")
print(df_clean.isnull().sum().sum())

In [ ]:
# Create Price Class (Target Variable 1)
def classify_price(price):
    if price < 10000:
        return 0  # Entry Level
    elif price < 25000:
        return 1  # Mid Range
    elif price < 50000:
        return 2  # High End
    else:
        return 3  # Flagship

df_clean['price_class'] = df_clean['price'].apply(classify_price)

# Price class mapping
price_class_names = {
    0: 'Entry Level',
    1: 'Mid Range',
    2: 'High End',
    3: 'Flagship'
}

print("Price Class Distribution:")
print(df_clean['price_class'].value_counts().sort_index().rename(price_class_names))

In [ ]:
# Create Usage Type (Target Variable 2)
def classify_usage(row):
    # Gaming: High refresh rate, fast processor, lots of RAM
    is_gaming = (
        row['refresh_rate'] >= 120 and 
        row['processor_speed'] >= 2.8 and 
        row['ram_capacity'] >= 8
    )
    
    # Photography: High camera specs
    is_photography = (
        row['primary_camera_rear'] >= 64 and 
        row['primary_camera_front'] >= 32
    )
    
    # Business: NFC, good specs
    is_business = (
        row['has_nfc'] == True and 
        row['ram_capacity'] >= 6 and 
        row['internal_memory'] >= 128
    )
    
    # Classification logic
    if is_gaming:
        return 'Gaming'
    elif is_photography:
        return 'Photography'
    elif is_business:
        return 'Business'
    elif row['price'] < 15000 and row['battery_capacity'] >= 5000:
        return 'Daily Use'
    else:
        return 'All-Rounder'

df_clean['usage_type'] = df_clean.apply(classify_usage, axis=1)

print("\nUsage Type Distribution:")
print(df_clean['usage_type'].value_counts())

In [ ]:
# Visualize target distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Price Class distribution
price_counts = df_clean['price_class'].value_counts().sort_index()
colors = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']
axes[0].bar([price_class_names[i] for i in price_counts.index], price_counts.values, color=colors)
axes[0].set_xlabel('Price Class')
axes[0].set_ylabel('Count')
axes[0].set_title('Price Class Distribution')
axes[0].tick_params(axis='x', rotation=45)

# Usage Type distribution
usage_counts = df_clean['usage_type'].value_counts()
usage_colors = ['#9b59b6', '#1abc9c', '#e67e22', '#34495e', '#2980b9']
axes[1].bar(usage_counts.index, usage_counts.values, color=usage_colors[:len(usage_counts)])
axes[1].set_xlabel('Usage Type')
axes[1].set_ylabel('Count')
axes[1].set_title('Usage Type Distribution')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Feature Engineering

In [ ]:
# Select features for model
feature_columns = [
    'num_cores',
    'processor_speed',
    'ram_capacity',
    'internal_memory',
    'battery_capacity',
    'screen_size',
    'refresh_rate',
    'primary_camera_rear',
    'primary_camera_front',
    'resolution_width',
    'resolution_height',
    'has_5g',
    'has_nfc',
    'fast_charging_available',
    'num_rear_cameras'
]

# Convert boolean to int
df_clean['has_5g'] = df_clean['has_5g'].astype(int)
df_clean['has_nfc'] = df_clean['has_nfc'].astype(int)
df_clean['fast_charging_available'] = df_clean['fast_charging_available'].astype(int)

# Prepare features
X = df_clean[feature_columns].copy()

# Handle any remaining missing values
X = X.fillna(X.median())

print(f"Features shape: {X.shape}")
print(f"\nFeature columns: {feature_columns}")

In [ ]:
# Correlation matrix
plt.figure(figsize=(14, 10))
corr_matrix = X.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Prepare target variables
y_price = df_clean['price_class']

# Encode usage type
usage_encoder = LabelEncoder()
y_usage = usage_encoder.fit_transform(df_clean['usage_type'])

print(f"Price classes: {np.unique(y_price)}")
print(f"Usage classes: {usage_encoder.classes_}")

## 5. Model Training - Price Class

In [ ]:
# Split data for Price Class model
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(
    X, y_price, test_size=0.2, random_state=42, stratify=y_price
)

print(f"Training set: {X_train_price.shape[0]} samples")
print(f"Test set: {X_test_price.shape[0]} samples")

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_price)
X_test_scaled = scaler.transform(X_test_price)

In [ ]:
# Hyperparameter tuning with GridSearchCV (Anti-Overfitting)
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [8, 10, 12],  # Limited depth to prevent overfitting
    'min_samples_split': [5, 10],  # Minimum samples to split
    'min_samples_leaf': [2, 4],  # Minimum samples in leaf
    'max_features': ['sqrt', 'log2']  # Limited features per split
}

print("Starting GridSearchCV for Price Class model...")
print("This may take a few minutes...\n")

rf_price = RandomForestClassifier(random_state=42, n_jobs=-1)

grid_search_price = GridSearchCV(
    rf_price, 
    param_grid, 
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search_price.fit(X_train_scaled, y_train_price)

print(f"\nBest Parameters: {grid_search_price.best_params_}")
print(f"Best CV Score: {grid_search_price.best_score_:.4f}")

In [ ]:
# Use best model
best_rf_price = grid_search_price.best_estimator_

# Predictions
y_train_pred = best_rf_price.predict(X_train_scaled)
y_test_pred = best_rf_price.predict(X_test_scaled)

# Check for overfitting
train_accuracy = accuracy_score(y_train_price, y_train_pred)
test_accuracy = accuracy_score(y_test_price, y_test_pred)

print("="*50)
print("PRICE CLASS MODEL EVALUATION")
print("="*50)
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Difference: {train_accuracy - test_accuracy:.4f}")

if (train_accuracy - test_accuracy) < 0.05:
    print("\n✅ Model is NOT overfitting (difference < 5%)")
else:
    print("\n⚠️ Model might be overfitting")

In [ ]:
# Cross-validation scores
cv_scores = cross_val_score(best_rf_price, X_train_scaled, y_train_price, cv=5)
print(f"\nCross-Validation Scores: {cv_scores}")
print(f"Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

In [ ]:
# Classification Report
print("\nClassification Report (Price Class):")
print("="*50)
target_names = ['Entry Level', 'Mid Range', 'High End', 'Flagship']
print(classification_report(y_test_price, y_test_pred, target_names=target_names))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test_price, y_test_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Price Class Model')
plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': best_rf_price.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
plt.xlabel('Importance')
plt.title('Feature Importance - Price Class Model')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 5 Important Features:")
print(feature_importance.head())

## 6. Model Training - Usage Type

In [ ]:
# Split data for Usage Type model
X_train_usage, X_test_usage, y_train_usage, y_test_usage = train_test_split(
    X, y_usage, test_size=0.2, random_state=42, stratify=y_usage
)

# Scale features
X_train_usage_scaled = scaler.transform(X_train_usage)
X_test_usage_scaled = scaler.transform(X_test_usage)

In [ ]:
# Train Usage Type model with best params (similar approach)
print("Training Usage Type model...")

rf_usage = RandomForestClassifier(
    n_estimators=150,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

rf_usage.fit(X_train_usage_scaled, y_train_usage)

# Evaluate
y_train_usage_pred = rf_usage.predict(X_train_usage_scaled)
y_test_usage_pred = rf_usage.predict(X_test_usage_scaled)

train_acc_usage = accuracy_score(y_train_usage, y_train_usage_pred)
test_acc_usage = accuracy_score(y_test_usage, y_test_usage_pred)

print("="*50)
print("USAGE TYPE MODEL EVALUATION")
print("="*50)
print(f"Training Accuracy: {train_acc_usage:.4f}")
print(f"Test Accuracy: {test_acc_usage:.4f}")
print(f"Difference: {train_acc_usage - test_acc_usage:.4f}")

In [ ]:
# Classification Report for Usage Type
print("\nClassification Report (Usage Type):")
print("="*50)
print(classification_report(y_test_usage, y_test_usage_pred, 
                           target_names=usage_encoder.classes_))

In [ ]:
# Confusion Matrix - Usage Type
plt.figure(figsize=(10, 8))
cm_usage = confusion_matrix(y_test_usage, y_test_usage_pred)
sns.heatmap(cm_usage, annot=True, fmt='d', cmap='Greens',
            xticklabels=usage_encoder.classes_, yticklabels=usage_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Usage Type Model')
plt.tight_layout()
plt.show()

## 7. Save Models

In [ ]:
import os

# Create model directory if not exists
model_dir = '../model'
os.makedirs(model_dir, exist_ok=True)

# Save Price Class model
joblib.dump(best_rf_price, f'{model_dir}/smartphone_price_model.pkl')
print("✅ Price Class model saved!")

# Save Usage Type model
joblib.dump(rf_usage, f'{model_dir}/smartphone_usage_model.pkl')
print("✅ Usage Type model saved!")

# Save Scaler
joblib.dump(scaler, f'{model_dir}/scaler.pkl')
print("✅ Scaler saved!")

# Save feature list
joblib.dump(feature_columns, f'{model_dir}/features.pkl')
print("✅ Feature list saved!")

# Save usage encoder
joblib.dump(usage_encoder, f'{model_dir}/usage_encoder.pkl')
print("✅ Usage encoder saved!")

# Save price class names
joblib.dump(price_class_names, f'{model_dir}/price_class_names.pkl')
print("✅ Price class names saved!")

## 8. Test Predictions

In [ ]:
# Test prediction function
def predict_smartphone(specs):
    """
    Predict price class and usage type for a smartphone
    
    specs: dict with feature values
    """
    # Create DataFrame
    input_df = pd.DataFrame([specs], columns=feature_columns)
    
    # Scale
    input_scaled = scaler.transform(input_df)
    
    # Predict
    price_pred = best_rf_price.predict(input_scaled)[0]
    price_proba = best_rf_price.predict_proba(input_scaled)[0]
    
    usage_pred = rf_usage.predict(input_scaled)[0]
    usage_proba = rf_usage.predict_proba(input_scaled)[0]
    
    return {
        'price_class': price_class_names[price_pred],
        'price_confidence': price_proba[price_pred] * 100,
        'usage_type': usage_encoder.classes_[usage_pred],
        'usage_confidence': usage_proba[usage_pred] * 100
    }

# Test with sample data
test_specs = {
    'num_cores': 8,
    'processor_speed': 3.2,
    'ram_capacity': 12,
    'internal_memory': 256,
    'battery_capacity': 5000,
    'screen_size': 6.7,
    'refresh_rate': 120,
    'primary_camera_rear': 108,
    'primary_camera_front': 32,
    'resolution_width': 1440,
    'resolution_height': 3200,
    'has_5g': 1,
    'has_nfc': 1,
    'fast_charging_available': 1,
    'num_rear_cameras': 3
}

result = predict_smartphone(test_specs)
print("\n" + "="*50)
print("TEST PREDICTION")
print("="*50)
print(f"Price Class: {result['price_class']} ({result['price_confidence']:.1f}% confidence)")
print(f"Usage Type: {result['usage_type']} ({result['usage_confidence']:.1f}% confidence)")

In [ ]:
# Test with entry level phone
entry_specs = {
    'num_cores': 4,
    'processor_speed': 1.8,
    'ram_capacity': 3,
    'internal_memory': 32,
    'battery_capacity': 5000,
    'screen_size': 6.5,
    'refresh_rate': 60,
    'primary_camera_rear': 13,
    'primary_camera_front': 5,
    'resolution_width': 720,
    'resolution_height': 1600,
    'has_5g': 0,
    'has_nfc': 0,
    'fast_charging_available': 0,
    'num_rear_cameras': 2
}

result_entry = predict_smartphone(entry_specs)
print("\nEntry Level Phone Prediction:")
print(f"Price Class: {result_entry['price_class']} ({result_entry['price_confidence']:.1f}%)")
print(f"Usage Type: {result_entry['usage_type']} ({result_entry['usage_confidence']:.1f}%)")

## 9. Summary

In [ ]:
print("\n" + "="*60)
print("MODEL TRAINING SUMMARY")
print("="*60)
print(f"\nDataset: {len(df_clean)} smartphones")
print(f"Features: {len(feature_columns)} features")
print(f"\n--- Price Class Model ---")
print(f"Algorithm: Random Forest Classifier")
print(f"Best Parameters: {grid_search_price.best_params_}")
print(f"Test Accuracy: {test_accuracy:.2%}")
print(f"CV Score: {cv_scores.mean():.2%}")
print(f"\n--- Usage Type Model ---")
print(f"Test Accuracy: {test_acc_usage:.2%}")
print(f"\n--- Saved Files ---")
print("- model/smartphone_price_model.pkl")
print("- model/smartphone_usage_model.pkl")
print("- model/scaler.pkl")
print("- model/features.pkl")
print("- model/usage_encoder.pkl")
print("- model/price_class_names.pkl")
print("\n✅ Training completed successfully!")